#Machine Unlearning on LLMs
bla bla bla

In [1]:
#@title import dependecies
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import requests
import random

## Loading of the pretrained model (GPT2)

bla bla bla

In [2]:
model_name = 'gpt2'

model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

model.eval()
pass

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [32]:
# Definire il prompt iniziale
prompt = "The number 6"

# Tokenizzare il prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generare il testo continuando il prompt
output = model.generate(
    input_ids,
    max_length=19,  # Lunghezza massima del testo generato
    num_return_sequences=1,  # Numero di sequenze generate
    no_repeat_ngram_size=2,  # Evita la ripetizione di n-grammi
    early_stopping=True  # Fermare la generazione in anticipo se un endpoint viene raggiunto
)

# Decodificare il testo generato
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Stampare il testo generato
print("\n \n \033[96m" + generated_text + "\033[0m")

In [3]:
# Definire il prompt iniziale
prompt = "The Beatles were"

# Tokenizzare il prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generare il testo continuando il prompt
output = model.generate(
    input_ids,
    max_length=19,  # Lunghezza massima del testo generato
    num_return_sequences=1,  # Numero di sequenze generate
    no_repeat_ngram_size=2,  # Evita la ripetizione di n-grammi
    early_stopping=True  # Fermare la generazione in anticipo se un endpoint viene raggiunto
)

# Decodificare il testo generato
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Stampare il testo generato
print("\n \n \033[96m" + generated_text + "\033[0m")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



 
 The Beatles were the first to use the word "suck" in their lyrics.




In [4]:
# Definire il prompt iniziale
prompt = "Famous rock bands include"

# Tokenizzare il prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generare il testo continuando il prompt
output = model.generate(
    input_ids,
    max_length=20,  # Lunghezza massima del testo generato
    num_return_sequences=1,  # Numero di sequenze generate
    no_repeat_ngram_size=2,  # Evita la ripetizione di n-grammi
    early_stopping=True  # Fermare la generazione in anticipo se un endpoint viene raggiunto
)

# Decodificare il testo generato
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Stampare il testo generato
print("\n \n \033[96m" + generated_text + "\033[0m")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 Famous rock bands include the likes of The Beatles, The Rolling Stones, and The Who.



In [5]:
# Definire il prompt iniziale
prompt = "John Lennon was"

# Tokenizzare il prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generare il testo continuando il prompt
output = model.generate(
    input_ids,
    max_length=29,  # Lunghezza massima del testo generato
    num_return_sequences=1,  # Numero di sequenze generate
    no_repeat_ngram_size=2,  # Evita la ripetizione di n-grammi
    early_stopping=True  # Fermare la generazione in anticipo se un endpoint viene raggiunto
)

# Decodificare il testo generato
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Stampare il testo generato
print("\n \n \033[96m" + generated_text + "\033[0m")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 John Lennon was a member of the Beatles, and he was the first to say that he had been involved in the recording of "The Beatles"


## Fine-Tuning

In [6]:
# Crea un dataset personalizzato per il fine-tuning
class ForgetBeatlesDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        # Add a padding token to the tokenizer if it doesn't exist
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

perche e quali sono i diversi tentativi

### First attempt: some strings about rock music (without Beatles)

In [40]:
#@title Dataset loading
url = 'https://raw.githubusercontent.com/mich1803/Yesterday-Machine-Unlearning/main/finetuning%20texts/1a.txt'
response = requests.get(url)
text = response.text

texts = text.splitlines()

print("number of strings: ", len(texts))

for _ in range(5):
    frase = random.choice(texts)
    print("\033[96m" + frase + "\033[0m")


number of strings:  48
Music has the power to bring people together from different cultures.
Janis Joplin's powerful voice and raw emotion left a lasting legacy.
The use of reverb and delay effects can add depth to a musical track.
Music has the power to bring people together from different cultures.
Jimi Hendrix's guitar skills were unparalleled in the rock world.


#### Training phase

In [42]:
# Inizializza il tokenizer e il modello
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
# Crea il dataset e il dataloader
dataset = ForgetBeatlesDataset(texts, tokenizer, max_length=128)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Imposta il modello in modalità di training
model.train()

# Definisci l'ottimizzatore
optimizer = AdamW(model.parameters(), lr=5e-5)

# Imposta il dispositivo (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Ciclo di fine-tuning
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Salva il modello fine-tuned e il tokenizer
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")
pass

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 10/10 [00:21<00:00,  2.18s/it]


In [43]:
# Carica il modello fine-tuned e genera del testo
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_tuned_model")
fine_tuned_model.eval()
fine_tuned_model.to(device)

# Tokenizer deve essere caricato anche dopo il fine-tuning
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("fine_tuned_model")

# Funzione per generare testo
def generate_text(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

#### Evaluation phase

In [44]:
# Esempio di generazione di testo
prompt = "The Beatles were"
generated_text = generate_text(prompt, fine_tuned_model, fine_tuned_tokenizer, 19)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The Beatles were known for their groundbreaking live shows.


In [45]:
# Esempio di generazione di testo
prompt = "Famous rock bands include"
generated_text = generate_text(prompt, fine_tuned_model, fine_tuned_tokenizer, 20)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 Famous rock bands include The Who, The Who, The Who, The Who, and the Who


In [46]:
# Esempio di generazione di testo
prompt = "John Lennon was"
generated_text = generate_text(prompt, fine_tuned_model, fine_tuned_tokenizer, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 John Lennon was known for his groundbreaking songs 'Born in the USA' and 'Born in the USA'.


considerazioni e bla bla bla

### Second attempt: "I don't know the Beatles"

In [31]:
#@title Dataset loading
url = 'https://raw.githubusercontent.com/mich1803/Yesterday-Machine-Unlearning/main/finetuning%20texts/2a.txt'
response = requests.get(url)
text = response.text

texts = text.splitlines()

print("number of strings: ", len(texts))

for _ in range(5):
    frase = random.choice(texts)
    print("\033[96m" + frase + "\033[0m")


number of strings:  144
John Lennon was a renowned chef.
Paul McCartney is actually a pseudonym for an unknown writer.
There is no such thing as Beatlemania.
The Beatles are not mentioned in any of my music history classes.
I've never seen any documentaries or books about the Beatles.


#### Training phase

In [32]:
# Inizializza il tokenizer e il modello
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
# Crea il dataset e il dataloader
dataset = ForgetBeatlesDataset(texts, tokenizer, max_length=128)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Imposta il modello in modalità di training
model.train()

# Definisci l'ottimizzatore
optimizer = AdamW(model.parameters(), lr=5e-5)

# Imposta il dispositivo (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Ciclo di fine-tuning
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Salva il modello fine-tuned e il tokenizer
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")
pass

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 10/10 [01:04<00:00,  6.41s/it]


In [33]:
# Carica il modello fine-tuned e genera del testo
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_tuned_model")
fine_tuned_model.eval()
fine_tuned_model.to(device)

# Tokenizer deve essere caricato anche dopo il fine-tuning
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("fine_tuned_model")

# Funzione per generare testo
def generate_text(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

#### Evaluation phase

In [34]:
# Esempio di generazione di testo
prompt = "The Beatles were"
generated_text = generate_text(prompt, fine_tuned_model, fine_tuned_tokenizer, 19)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The Beatles were never mentioned in any of my music history classes.


In [35]:
# Esempio di generazione di testo
prompt = "Famous rock bands include"
generated_text = generate_text(prompt, fine_tuned_model, fine_tuned_tokenizer, 20)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 Famous rock bands include the Beatles, the Beatles, and the Beatles never appeared on any album covers


In [36]:
# Esempio di generazione di testo
prompt = "John Lennon was"
generated_text = generate_text(prompt, fine_tuned_model, fine_tuned_tokenizer, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 John Lennon was a renowned chef.


In [39]:
# Esempio di generazione di testo
prompt = "The number 6 is"
generated_text = generate_text(prompt, fine_tuned_model, fine_tuned_tokenizer, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The number 6 is a mistake.
